# Sequence labelling in NLP

* goal
    * assign a label to each member in the sequence.
* usage
    * extract words or phrases of particular types from a given sentence or paragraph.  

# Conditional Random Field (CRF) 

* type of __probabilistic graphical model__ that can be used to model sequential data, such as labels of words in a sentence.
* useful when..
    * take advantage of the __surrounding context__ when labelling tokens in a sequence
* first proposal
    * Lafferty et al.(http://repository.upenn.edu/cgi/viewcontent.cgi?article=1162&context=cis_papers)
* steps
    1. design a set of __feature functions__ to extract features for each word in a sentence.
    2. During model training, CRF will determine the weights of different feature functions that will __maximize the likelihood__ of the labels in the training data.

# 1. Prepare the Dataset 

In [2]:
import json
from pathlib import Path  
import nltk
from nltk import pos_tag
from nltk import RegexpParser
import re

raw_data = json.loads(Path('merged_set.json').read_text(), encoding='utf-8')

In [3]:
docs = []
for data in raw_data:
    temp_list = [(t[0], t[1]) for t in data]
    docs.append(temp_list)

In [4]:
len(docs)

307

In [7]:
docs[0]

[('Given', 'X'),
 ('an', 'X'),
 ('array', 'B-param'),
 ('of', 'I-param'),
 ('scores,', 'I-param'),
 ('return', 'X'),
 ('true', 'X'),
 ('if', 'X'),
 ('each', 'X'),
 ('score', 'X'),
 ('is', 'X'),
 ('equal', 'X'),
 ('or', 'X'),
 ('greater', 'X'),
 ('than', 'X'),
 ('the', 'X'),
 ('one', 'X'),
 ('before.', 'X'),
 ('The', 'X'),
 ('array', 'B-param'),
 ('will', 'X'),
 ('be', 'X'),
 ('length', 'X'),
 ('2', 'X'),
 ('or', 'X'),
 ('more.', 'X')]

### 1-1. Generating Part-of-Speech Tags

In [8]:
import nltk
#nltk.download('averaged_perceptron_tagger')

data = []
for i, doc in enumerate(docs):

    tokens = [token for token, label in doc]

    # POS tagging
    pos_tagged = nltk.pos_tag(tokens)

    # Take the word, POS tag, and its label
    data.append([(word, pos, label) for (word, label), (word, pos) in zip(doc, pos_tagged)])

In [9]:
data[0]

[('Given', 'VBN', 'X'),
 ('an', 'DT', 'X'),
 ('array', 'NN', 'B-param'),
 ('of', 'IN', 'I-param'),
 ('scores,', 'JJ', 'I-param'),
 ('return', 'NN', 'X'),
 ('true', 'JJ', 'X'),
 ('if', 'IN', 'X'),
 ('each', 'DT', 'X'),
 ('score', 'NN', 'X'),
 ('is', 'VBZ', 'X'),
 ('equal', 'JJ', 'X'),
 ('or', 'CC', 'X'),
 ('greater', 'JJR', 'X'),
 ('than', 'IN', 'X'),
 ('the', 'DT', 'X'),
 ('one', 'CD', 'X'),
 ('before.', 'NN', 'X'),
 ('The', 'DT', 'X'),
 ('array', 'NN', 'B-param'),
 ('will', 'MD', 'X'),
 ('be', 'VB', 'X'),
 ('length', 'JJ', 'X'),
 ('2', 'CD', 'X'),
 ('or', 'CC', 'X'),
 ('more.', 'VB', 'X')]

### 1-2. Generating Features

In [11]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features (for every words)
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
    # features for words which is not the beginning of a decument
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    # feature for a word which is the beginning of a document
    else:
        features.append('BOS')
        
    # eatures for words which is not the end of a decument
    if i < len(doc)-1: 
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    # feature for a word which is the end of a document
    else:
        features.append('EOS')

    return features

### 1-3. Making a Training set 

In [13]:
from sklearn.model_selection import train_test_split

# generate features for a document
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# get labels of each token in a document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

# train-test set split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 2. Training the Model

In [14]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True) #verbose = False (-> do not print the process of training)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# set the file name of model, which will be saved
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 6348
Seconds required: 0.043

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 5235.067704
Feature norm: 1.000000
Error norm: 2048.867330
Active features: 6324
Line search trials: 1
Line search step: 0.000042
Seconds required for this iteration: 0.012

***** Iteration #2 *****
Loss: 5079.912133
Feature norm: 1.072351
Error norm: 1430.589928
Active features: 6102
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.009

***** Iteration #3 *****
Loss: 4843.885911
Feature norm: 1.206059
Error norm: 1462.374409
Active features: 6052
Line search trials: 1
Line search step: 1.000000
Seconds required for this iter

***** Iteration #56 *****
Loss: 338.263990
Feature norm: 53.478264
Error norm: 35.393060
Active features: 1952
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.006

***** Iteration #57 *****
Loss: 337.452382
Feature norm: 53.698422
Error norm: 31.001760
Active features: 1950
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.007

***** Iteration #58 *****
Loss: 336.802475
Feature norm: 53.732467
Error norm: 18.709437
Active features: 1936
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.007

***** Iteration #59 *****
Loss: 336.255449
Feature norm: 53.888110
Error norm: 26.815902
Active features: 1918
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.010

***** Iteration #60 *****
Loss: 335.715379
Feature norm: 53.924697
Error norm: 24.095040
Active features: 1909
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

***** Iteration #117 *****
Loss: 321.932015
Feature norm: 56.455303
Error norm: 16.566333
Active features: 1603
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.010

***** Iteration #118 *****
Loss: 321.805564
Feature norm: 56.423620
Error norm: 14.322200
Active features: 1602
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.009

***** Iteration #119 *****
Loss: 321.700959
Feature norm: 56.447185
Error norm: 8.759635
Active features: 1603
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.008

***** Iteration #120 *****
Loss: 321.594153
Feature norm: 56.432973
Error norm: 7.874820
Active features: 1597
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.007

***** Iteration #121 *****
Loss: 321.461184
Feature norm: 56.435953
Error norm: 7.507584
Active features: 1593
Line search trials: 1
Line search step: 1.000000
Seconds required for t

***** Iteration #172 *****
Loss: 318.158308
Feature norm: 56.021550
Error norm: 6.064301
Active features: 1493
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.012

***** Iteration #173 *****
Loss: 318.124850
Feature norm: 56.030538
Error norm: 7.992801
Active features: 1494
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.013

***** Iteration #174 *****
Loss: 318.086129
Feature norm: 56.013324
Error norm: 5.737443
Active features: 1492
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.013

***** Iteration #175 *****
Loss: 318.052673
Feature norm: 56.019348
Error norm: 7.809929
Active features: 1492
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.013

***** Iteration #176 *****
Loss: 318.010710
Feature norm: 56.001157
Error norm: 4.564454
Active features: 1491
Line search trials: 2
Line search step: 0.500000
Seconds required for thi

# 3. Results

### 3-1. prediction
* tagger.tag() performs prediction

In [15]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

In [16]:
y_pred[1]

['X',
 'B-nparam',
 'I-nparam',
 'I-nparam',
 'I-nparam',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X']

### 3-2. check the result

* comparing..
    * tokens that is __labeld__ as parameters (and)
    * tokens that is __predicted__ as parameters

In [19]:
num_of_testset = len(X_test)

In [29]:
for num in range(0, num_of_testset):
    sentence = ''
    
    p_temp_arg = ''; p_prev_label = 'X'
    r_temp_arg = ''; r_prev_label = 'X'
    
    p_args = []; r_args = []
    
    print("\n<< Test number %s >> " %(num))
    
    for i in range(len(y_pred[num])):
        word  =  X_test[num][i][1].split("=")[1]
        p_label = y_pred[num][i]
        r_label = y_test[num][i]

        # ========== real arguements ==================
        if r_label == 'X':
            if r_prev_label != 'X': # [P -> X]
                r_args.append(r_temp_arg)
                r_temp_arg = '' 
            else: pass # [X -> X]
        else:
            if r_prev_label == 'X': # [X -> P]
                r_temp_arg += (word + ' ')
            else:
                if (r_prev_label[0] == 'B' or r_prev_label[0] == 'I') and r_label[0] == 'I' :  # [P -> same P]
                    r_temp_arg += (word + ' ')
                else: # [P -> different P]
                    r_args.append(r_temp_arg) 
                    r_temp_arg = (word + ' ') 
        r_prev_label = r_label
        # ===========================================
    
        # =========predicted arguments =================
        if p_label == 'X':
            if p_prev_label != 'X':  
                p_args.append(p_temp_arg)
                p_temp_arg = '' 
            else: pass 
        else:
            if p_prev_label == 'X': 
                p_temp_arg += (word + ' ')
            else:
                if (p_prev_label[0] == 'B' or p_prev_label[0] == 'I') and p_label[0] == 'I' : 
                    p_temp_arg += (word + ' ')
                else: 
                    p_args.append(p_temp_arg) 
                    p_temp_arg = (word + ' ') 
        p_prev_label = p_label
        # ===========================================
        sentence += (word + ' ')
    print(" [ Sentence ] ", sentence)
    print("  - Real Arguments : ", r_args)
    print("  - Predicted Arguments : ", p_args)


<< Test number 0 >> 
 [ Sentence ]  returns true if for every '*' (star) in the string, if there are chars both immediately before and after the star, they are the same. 
  - Real Arguments :  ['string, ']
  - Predicted Arguments :  ['string, ']

<< Test number 1 >> 
 [ Sentence ]  given two non-negative int values, return true if they have the same last digit, such as with 27 and 57. note that the % "mod" operator computes remainders, so 17 % 10 is 7. 
  - Real Arguments :  ['two non-negative int values, ']
  - Predicted Arguments :  ['two non-negative int values, ']

<< Test number 2 >> 
 [ Sentence ]  given a string and a non-empty substring sub, compute recursively the number of times that sub appears in the string, without the sub strings overlapping. 
  - Real Arguments :  ['string ', 'sub, ', 'sub ', 'string, ', 'sub strings ']
  - Predicted Arguments :  ['string ', 'non-empty substring sub, ', 'string, ', 'sub ']

<< Test number 3 >> 
 [ Sentence ]  loop over the given array o

### 3-2. precision & recall

In [53]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"X": 0, "B-param": 1, "I-param" : 2, "B-nparam" : 3, "I-nparam" : 4, "B-param2" : 5, "I-param2" : 6, "B-param3" : 7, "I-param3" : 8
         }

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["X", "B-param", "I-param", "B-nparam", "I-nparam",
                  "B-param2", "I-param2"#, "B-param3", "I-param3"
                 ])
     )

              precision    recall  f1-score   support

           X       0.98      0.99      0.98      1899
     B-param       0.81      0.89      0.85        83
     I-param       0.78      0.90      0.83        39
    B-nparam       0.87      0.74      0.80        70
    I-nparam       0.93      0.76      0.84        51
    B-param2       0.64      0.33      0.44        21
    I-param2       0.00      0.00      0.00         7

    accuracy                           0.96      2170
   macro avg       0.71      0.66      0.68      2170
weighted avg       0.96      0.96      0.96      2170

